In [1]:
# %%
"""
# 🔄 CNN K-Fold Cross-Validation
Visual Intelligence Project - DeepLIFT Assignment
Phase 1: Data & CNN Foundation (Day 4)

**Objective**: Implement 5-fold cross-validation for CNN to get robust performance metrics
**Requirements**: Mean accuracy and F1 scores, statistical analysis, model validation
**Target**: Prove CNN consistently achieves >70% accuracy across all folds
"""

# %%
# =============================================================================
# 📦 IMPORTS AND SETUP
# =============================================================================
import torch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Disable compilation features that cause issues
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='torch')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support
import scipy.stats as stats
from tqdm import tqdm
import json
import time
from pathlib import Path
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'🖥️  Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')



🖥️  Using device: cuda
GPU: NVIDIA GeForce RTX 4070 Laptop GPU
GPU Memory: 8.6 GB


In [2]:
# %%
# =============================================================================
# 🗂️ LOAD CONFIGURATION AND SETUP PATHS
# =============================================================================

# Define paths robustly relative to the notebook location
NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent
CONFIG_PATH = PROJECT_ROOT / "config.json"

# Load configuration from previous notebooks
if CONFIG_PATH.exists():
    with open(CONFIG_PATH, 'r') as f:
        config = json.load(f)
    print("✅ Configuration loaded from previous notebooks")
else:
    print("⚠️  Configuration not found, using default paths")
    config = {
        "dataset": {"processed_path": str(PROJECT_ROOT / "data" / "processed")},
        "paths": {
            "project_root": str(PROJECT_ROOT),
            "models": str(PROJECT_ROOT / "models"),
            "results": str(PROJECT_ROOT / "results")
        }
    }

# Setup paths
PROCESSED_DATA_PATH = Path(config["dataset"]["processed_path"])
MODELS_PATH = Path(config["paths"]["models"])
RESULTS_PATH = Path(config["paths"]["results"])

# Create K-fold specific results directory
KFOLD_RESULTS_PATH = RESULTS_PATH / "kfold"
KFOLD_RESULTS_PATH.mkdir(parents=True, exist_ok=True)

print("📁 Project Paths:")
print(f"Processed Data: {PROCESSED_DATA_PATH}")
print(f"Models: {MODELS_PATH}")
print(f"Results: {RESULTS_PATH}")
print(f"K-Fold Results: {KFOLD_RESULTS_PATH}")



✅ Configuration loaded from previous notebooks
📁 Project Paths:
Processed Data: D:\University\4th Semester\4. Visual Intelligence\Project\data\processed
Models: D:\University\4th Semester\4. Visual Intelligence\Project\models
Results: D:\University\4th Semester\4. Visual Intelligence\Project\results
K-Fold Results: D:\University\4th Semester\4. Visual Intelligence\Project\results\kfold


In [3]:
# %%
# =============================================================================
# 🏗️ MODEL CLASSES (IDENTICAL TO PREVIOUS NOTEBOOKS)
# =============================================================================

class SharedClassifier(nn.Module):
    """Shared classifier for CNN and ScatNet - identical to previous notebooks"""
    
    def __init__(self, input_features, num_classes=2, dropout_rate=0.5):
        super(SharedClassifier, self).__init__()
        
        self.input_features = input_features
        self.num_classes = num_classes
        self.dropout_rate = dropout_rate
        
        self.classifier = nn.Sequential(
            # First fully connected layer
            nn.Linear(input_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            
            # Second fully connected layer
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            
            # Third fully connected layer
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate / 2),
            
            # Output layer
            nn.Linear(128, num_classes)
        )
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialize model weights"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        return self.classifier(x)

class LungCancerCNN(nn.Module):
    """Enhanced CNN model - identical to previous notebooks"""
    
    def __init__(self, num_classes=2, dropout_rate=0.5, input_channels=3):
        super(LungCancerCNN, self).__init__()
        
        self.num_classes = num_classes
        self.dropout_rate = dropout_rate
        self.input_channels = input_channels
        
        # Feature extraction layers
        self.features = nn.Sequential(
            # First convolutional block (3 -> 32)
            nn.Conv2d(input_channels, 32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            # Second convolutional block (32 -> 64)
            nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            # Third convolutional block (64 -> 128)
            nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            # Fourth convolutional block (128 -> 256)
            nn.Conv2d(128, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            # Fifth convolutional block (256 -> 512)
            nn.Conv2d(256, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((2, 2)),
            nn.Dropout2d(0.25),
        )
        
        # Calculate feature size
        self.feature_size = 512 * 2 * 2
        
        # Shared classifier
        self.classifier = SharedClassifier(
            input_features=self.feature_size,
            num_classes=num_classes,
            dropout_rate=dropout_rate
        )
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialize model weights"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def get_features(self, x):
        """Extract features before classification"""
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return x



In [4]:
# %%
# =============================================================================
# 📊 ENHANCED DATASET CLASS
# =============================================================================

class LungCancerDataset(Dataset):
    """Enhanced dataset class for K-fold validation"""
    
    def __init__(self, data_dir, split='train', transform=None, class_to_idx=None):
        self.data_dir = Path(data_dir) / split
        self.transform = transform
        self.split = split
        self.classes = ['adenocarcinoma', 'benign']
        self.class_to_idx = class_to_idx or {'adenocarcinoma': 0, 'benign': 1}
        self.samples = []
        self.class_counts = {cls: 0 for cls in self.classes}
        
        self._load_samples()
        
        if len(self.samples) > 0:
            print(f"📊 {split.capitalize()} dataset: {len(self.samples)} images")
            self._print_class_distribution()
        else:
            print(f"⚠️  No images found in {self.data_dir}")
    
    def _load_samples(self):
        """Load all image paths with labels"""
        for class_name in self.classes:
            class_dir = self.data_dir / class_name
            if class_dir.exists():
                extensions = ['*.jpeg', '*.jpg', '*.png', '*.bmp']
                for ext in extensions:
                    for img_path in class_dir.glob(ext):
                        self.samples.append((str(img_path), self.class_to_idx[class_name]))
                        self.class_counts[class_name] += 1
    
    def _print_class_distribution(self):
        """Print class distribution"""
        total = len(self.samples)
        print(f"  Class distribution:")
        for class_name in self.classes:
            count = self.class_counts[class_name]
            percentage = (count / total) * 100 if total > 0 else 0
            print(f"    {class_name.capitalize()}: {count} ({percentage:.1f}%)")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"⚠️  Error loading {img_path}: {e}")
            image = Image.new('RGB', (224, 224), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
            
        return image, label
    
    def get_labels(self):
        """Get all labels for stratified splitting"""
        return [sample[1] for sample in self.samples]



In [5]:
# %%
# =============================================================================
# 🎨 DATA TRANSFORMS
# =============================================================================

def get_kfold_transforms(augment_strength='light'):
    """Get transforms optimized for K-fold validation"""
    
    if augment_strength == 'light':
        train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=10),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    elif augment_strength == 'medium':
        train_transform = transforms.Compose([
            transforms.Resize((240, 240)),
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.3),
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    else:  # strong
        train_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(degrees=20),
            transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.15),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    # Validation transform (no augmentation)
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform



In [6]:
# %%
# =============================================================================
# ⚙️ K-FOLD CONFIGURATION
# =============================================================================

class KFoldConfig:
    """Enhanced configuration for K-Fold Cross Validation"""
    
    def __init__(self):
        # K-Fold settings
        self.k_folds = 5
        self.random_state = 42
        self.use_stratified = True  # Maintain class distribution
        
        # Training settings
        self.batch_size = 32  # Smaller batch for stability
        self.learning_rate = 0.001
        self.num_epochs = 15 # Enough for convergence
        self.weight_decay = 1e-4
        
        # Early stopping
        self.patience = 5  # Early stopping patience
        self.min_delta = 0.001  # Minimum improvement
        
        # Data augmentation
        self.augment_strength = 'light'  # Conservative for K-fold
        
        # Model settings
        self.dropout_rate = 0.5
        
        # Other settings
        self.save_all_models = True
        self.detailed_logging = True
        
        print(f"📋 K-Fold Configuration:")
        print(f"   K-Folds: {self.k_folds}")
        print(f"   Stratified: {self.use_stratified}")
        print(f"   Epochs per fold: {self.num_epochs}")
        print(f"   Batch size: {self.batch_size}")
        print(f"   Learning rate: {self.learning_rate}")
        print(f"   Augmentation: {self.augment_strength}")

# Initialize configuration
kfold_config = KFoldConfig()



📋 K-Fold Configuration:
   K-Folds: 5
   Stratified: True
   Epochs per fold: 15
   Batch size: 32
   Learning rate: 0.001
   Augmentation: light


In [7]:
# %%
# =============================================================================
# 🔥 ENHANCED TRAINING FUNCTIONS
# =============================================================================

def train_single_fold_enhanced(model, train_loader, val_loader, fold_num, config, device=device):
    """Enhanced training for a single fold with better monitoring"""
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        model.parameters(), 
        lr=config.learning_rate, 
        weight_decay=config.weight_decay,
        betas=(0.9, 0.999)
    )
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3, min_lr=1e-6
    )
    
    # Training history
    fold_history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [], 'val_f1': [],
        'learning_rate': []
    }
    
    best_val_acc = 0.0
    patience_counter = 0
    best_model_state = None
    
    print(f"🔄 Training Fold {fold_num + 1}/{config.k_folds}")
    print("-" * 40)
    
    for epoch in range(config.num_epochs):
        epoch_start_time = time.time()
        
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_pbar = tqdm(train_loader, desc=f'Fold {fold_num+1} Epoch {epoch+1} [Train]', leave=False)
        for data, target in train_pbar:
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(output, 1)
            train_total += target.size(0)
            train_correct += (predicted == target).sum().item()
            
            train_pbar.set_postfix({'Loss': f'{loss.item():.4f}'})
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc=f'Fold {fold_num+1} Epoch {epoch+1} [Val]', leave=False)
            for data, target in val_pbar:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                
                val_loss += loss.item()
                _, predicted = torch.max(output, 1)
                val_total += target.size(0)
                val_correct += (predicted == target).sum().item()
                
                all_preds.extend(predicted.cpu().numpy())
                all_targets.extend(target.cpu().numpy())
                
                val_pbar.set_postfix({'Loss': f'{loss.item():.4f}'})
        
        # Calculate metrics
        epoch_train_loss = train_loss / len(train_loader)
        epoch_train_acc = 100.0 * train_correct / train_total
        epoch_val_loss = val_loss / len(val_loader)
        epoch_val_acc = 100.0 * val_correct / val_total
        epoch_val_f1 = f1_score(all_targets, all_preds, average='weighted')
        
        # Update learning rate scheduler
        scheduler.step(epoch_val_acc)
        
        # Save history
        fold_history['train_loss'].append(epoch_train_loss)
        fold_history['train_acc'].append(epoch_train_acc)
        fold_history['val_loss'].append(epoch_val_loss)
        fold_history['val_acc'].append(epoch_val_acc)
        fold_history['val_f1'].append(epoch_val_f1)
        fold_history['learning_rate'].append(optimizer.param_groups[0]['lr'])
        
        epoch_time = time.time() - epoch_start_time
        
        # Print progress
        if config.detailed_logging and ((epoch + 1) % 5 == 0 or epoch == config.num_epochs - 1):
            print(f"  Epoch {epoch+1:2d}: Train: {epoch_train_acc:6.2f}%, Val: {epoch_val_acc:6.2f}%, "
                  f"F1: {epoch_val_f1:.4f}, LR: {optimizer.param_groups[0]['lr']:.2e}, Time: {epoch_time:.1f}s")
        
        # Check for improvement
        if epoch_val_acc > best_val_acc + config.min_delta:
            best_val_acc = epoch_val_acc
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= config.patience:
            print(f"  Early stopping at epoch {epoch + 1} (no improvement for {config.patience} epochs)")
            break
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return {
        'best_val_acc': best_val_acc,
        'final_val_f1': fold_history['val_f1'][-1] if fold_history['val_f1'] else 0,
        'history': fold_history,
        'fold_num': fold_num,
        'epochs_trained': len(fold_history['train_loss']),
        'final_predictions': all_preds,
        'final_targets': all_targets
    }



In [8]:
# %%
# =============================================================================
# 🔄 MAIN K-FOLD IMPLEMENTATION
# =============================================================================

def run_enhanced_kfold_validation(config):
    """Run enhanced K-Fold Cross Validation with comprehensive analysis"""
    
    print("🚀 STARTING ENHANCED K-FOLD CROSS VALIDATION")
    print("=" * 60)
    
    # Load dataset
    train_transform, val_transform = get_kfold_transforms(config.augment_strength)
    full_dataset = LungCancerDataset(
        data_dir=PROCESSED_DATA_PATH,
        split='train',
        transform=None,  # We'll apply transforms per fold
        class_to_idx={'adenocarcinoma': 0, 'benign': 1}
    )
    
    if len(full_dataset) == 0:
        print("❌ No training data found!")
        return None, None
    
    # Get labels for stratified splitting
    all_labels = full_dataset.get_labels()
    label_counts = Counter(all_labels)
    print(f"\n📊 Dataset composition:")
    for label, count in label_counts.items():
        class_name = full_dataset.classes[label]
        print(f"   {class_name.capitalize()}: {count} samples")
    
    # Create K-Fold splitter
    if config.use_stratified:
        kfold = StratifiedKFold(
            n_splits=config.k_folds, 
            shuffle=True, 
            random_state=config.random_state
        )
        split_generator = kfold.split(range(len(full_dataset)), all_labels)
        print(f"✅ Using StratifiedKFold to maintain class distribution")
    else:
        kfold = KFold(
            n_splits=config.k_folds, 
            shuffle=True, 
            random_state=config.random_state
        )
        split_generator = kfold.split(range(len(full_dataset)))
        print(f"✅ Using standard KFold")
    
    # Results storage
    fold_results = []
    all_val_accs = []
    all_val_f1s = []
    all_fold_predictions = []
    all_fold_targets = []
    
    start_time = time.time()
    
    # Run K-Fold validation
    for fold_num, (train_indices, val_indices) in enumerate(split_generator):
        
        fold_start_time = time.time()
        
        print(f"\n{'='*50}")
        print(f"FOLD {fold_num + 1}/{config.k_folds}")
        print(f"{'='*50}")
        
        # Check class distribution in this fold
        train_labels = [all_labels[i] for i in train_indices]
        val_labels = [all_labels[i] for i in val_indices]
        
        train_label_counts = Counter(train_labels)
        val_label_counts = Counter(val_labels)
        
        print(f"📊 Fold {fold_num + 1} distribution:")
        print(f"   Train: {dict(train_label_counts)} (total: {len(train_indices)})")
        print(f"   Val:   {dict(val_label_counts)} (total: {len(val_indices)})")
        
        # Create datasets for this fold
        train_dataset = Subset(full_dataset, train_indices)
        val_dataset = Subset(full_dataset, val_indices)
        
        # Apply transforms by temporarily modifying the dataset
        original_transform = full_dataset.transform
        
        # Create data loaders with transforms
        full_dataset.transform = train_transform
        train_loader = DataLoader(
            train_dataset, 
            batch_size=config.batch_size, 
            shuffle=True, 
            num_workers=0,
            pin_memory=torch.cuda.is_available(),
            drop_last=True,
        )
        
        full_dataset.transform = val_transform
        val_loader = DataLoader(
            val_dataset, 
            batch_size=config.batch_size, 
            shuffle=False, 
            num_workers=0,
            pin_memory=torch.cuda.is_available(),

        )
        
        # Create fresh model for this fold
        model = LungCancerCNN(
            num_classes=2, 
            dropout_rate=config.dropout_rate
        ).to(device)
        
        print(f"📱 Model parameters: {sum(p.numel() for p in model.parameters()):,}")
        
        # Train the model
        fold_result = train_single_fold_enhanced(
            model, train_loader, val_loader, fold_num, config, device
        )
        
        # Restore original transform
        full_dataset.transform = original_transform
        
        # Save model if requested
        if config.save_all_models:
            model_path = MODELS_PATH / f"cnn_fold_{fold_num + 1}_best.pth"
            torch.save({
                'model_state_dict': model.state_dict(),
                'fold_num': fold_num + 1,
                'best_val_acc': fold_result['best_val_acc'],
                'config': config.__dict__,
                'train_indices': train_indices.tolist(),
                'val_indices': val_indices.tolist()
            }, model_path)
            print(f"💾 Model saved: {model_path}")
        
        fold_time = time.time() - fold_start_time
        
        # Store results
        fold_result['fold_time_minutes'] = fold_time / 60
        fold_results.append(fold_result)
        all_val_accs.append(fold_result['best_val_acc'])
        all_val_f1s.append(fold_result['final_val_f1'])
        all_fold_predictions.extend(fold_result['final_predictions'])
        all_fold_targets.extend(fold_result['final_targets'])
        
        print(f"✅ Fold {fold_num + 1} completed!")
        print(f"   Best Val Acc: {fold_result['best_val_acc']:.2f}%")
        print(f"   Final F1: {fold_result['final_val_f1']:.4f}")
        print(f"   Time: {fold_time/60:.1f} minutes")
        print(f"   Epochs: {fold_result['epochs_trained']}")
    
    total_time = time.time() - start_time
    
    # Calculate comprehensive statistics
    results_summary = calculate_kfold_statistics(
        all_val_accs, all_val_f1s, fold_results, total_time, config
    )
    
    # Save results
    save_kfold_results(results_summary, fold_results, all_fold_predictions, all_fold_targets)
    
    return results_summary, fold_results

def calculate_kfold_statistics(all_val_accs, all_val_f1s, fold_results, total_time, config):
    """Calculate comprehensive K-fold statistics"""
    
    # Basic statistics
    mean_acc = np.mean(all_val_accs)
    std_acc = np.std(all_val_accs, ddof=1)  # Sample standard deviation
    mean_f1 = np.mean(all_val_f1s)
    std_f1 = np.std(all_val_f1s, ddof=1)
    
    # Confidence intervals (95%)
    n = len(all_val_accs)
    t_value = stats.t.ppf(0.975, n-1)  # 95% confidence interval
    acc_ci_margin = t_value * std_acc / np.sqrt(n)
    f1_ci_margin = t_value * std_f1 / np.sqrt(n)
    
    # Performance assessment
    all_above_70 = all(acc >= 70 for acc in all_val_accs)
    consistency_level = 'EXCELLENT' if std_acc < 2 else 'GOOD' if std_acc < 5 else 'MODERATE'
    
    # Average training time per fold
    avg_fold_time = total_time / (60 * config.k_folds)
    
    results_summary = {
        'k_folds': config.k_folds,
        'accuracy_scores': all_val_accs,
        'f1_scores': all_val_f1s,
        'statistics': {
            'mean_accuracy': float(mean_acc),
            'std_accuracy': float(std_acc),
            'mean_f1': float(mean_f1),
            'std_f1': float(std_f1),
            'min_accuracy': float(min(all_val_accs)),
            'max_accuracy': float(max(all_val_accs)),
            'min_f1': float(min(all_val_f1s)),
            'max_f1': float(max(all_val_f1s)),
            'accuracy_ci_lower': float(mean_acc - acc_ci_margin),
            'accuracy_ci_upper': float(mean_acc + acc_ci_margin),
            'f1_ci_lower': float(mean_f1 - f1_ci_margin),
            'f1_ci_upper': float(mean_f1 + f1_ci_margin)
        },
        'performance_assessment': {
            'all_folds_above_70': all_above_70,
            'consistency_level': consistency_level,
            'target_achieved': mean_acc >= 70,
            'performance_grade': 'OUTSTANDING' if mean_acc > 95 else 'EXCELLENT' if mean_acc > 85 else 'GOOD' if mean_acc >= 70 else 'NEEDS_IMPROVEMENT'
        },
        'timing': {
            'total_time_minutes': total_time / 60,
            'average_fold_time_minutes': avg_fold_time,
            'total_epochs': sum(fold['epochs_trained'] for fold in fold_results)
        },
        'config': {
            'k_folds': config.k_folds,
            'use_stratified': config.use_stratified,
            'batch_size': config.batch_size,
            'learning_rate': config.learning_rate,
            'num_epochs': config.num_epochs,
            'augment_strength': config.augment_strength
        }
    }
    
    return results_summary

def save_kfold_results(results_summary, fold_results, all_predictions, all_targets):
    """Save K-fold results to files"""
    
    # Save main results
    results_file = KFOLD_RESULTS_PATH / "cnn_kfold_results.json"
    with open(results_file, 'w') as f:
        # Create a copy without non-serializable objects
        save_data = {
            'results_summary': results_summary,
            'fold_details': []
        }
        
        for fold_result in fold_results:
            fold_data = {
                'fold_num': fold_result['fold_num'],
                'best_val_acc': fold_result['best_val_acc'],
                'final_val_f1': fold_result['final_val_f1'],
                'epochs_trained': fold_result['epochs_trained'],
                'fold_time_minutes': fold_result['fold_time_minutes'],
                'history': fold_result['history']
            }
            save_data['fold_details'].append(fold_data)
        
        json.dump(save_data, f, indent=2)
    
    # Save predictions for further analysis
    predictions_file = KFOLD_RESULTS_PATH / "cnn_kfold_predictions.json"
    with open(predictions_file, 'w') as f:
        json.dump({
            'predictions': [int(p) for p in all_predictions],
            'targets': [int(t) for t in all_targets],
            'class_names': ['adenocarcinoma', 'benign']
        }, f, indent=2)
    
    print(f"💾 Results saved:")
    print(f"   Main results: {results_file}")
    print(f"   Predictions: {predictions_file}")



In [9]:
# %%
# =============================================================================
# 📊 ENHANCED VISUALIZATION FUNCTIONS
# =============================================================================

def create_comprehensive_kfold_visualizations(results_summary, fold_results):
    """Create comprehensive K-fold visualizations"""
    
    # Create figure with multiple subplots
    fig = plt.figure(figsize=(20, 16))
    gs = fig.add_gridspec(4, 4, hspace=0.3, wspace=0.3)
    
    # Main title
    fig.suptitle('CNN K-Fold Cross-Validation Comprehensive Analysis', 
                fontsize=20, fontweight='bold', y=0.98)
    
    # 1. Accuracy per fold (top-left)
    ax1 = fig.add_subplot(gs[0, 0])
    bars = ax1.bar(range(1, kfold_config.k_folds + 1), results_summary['accuracy_scores'], 
                   color='skyblue', alpha=0.8, edgecolor='navy', linewidth=2)
    ax1.axhline(y=results_summary['statistics']['mean_accuracy'], color='red', 
                linestyle='--', linewidth=2, label=f"Mean: {results_summary['statistics']['mean_accuracy']:.2f}%")
    ax1.axhline(y=70, color='green', linestyle='--', alpha=0.7, linewidth=2, label='70% Target')
    
    # Add confidence interval
    mean_acc = results_summary['statistics']['mean_accuracy']
    ci_lower = results_summary['statistics']['accuracy_ci_lower']
    ci_upper = results_summary['statistics']['accuracy_ci_upper']
    ax1.fill_between([0.5, kfold_config.k_folds + 0.5], ci_lower, ci_upper, 
                     alpha=0.2, color='red', label=f'95% CI')
    
    ax1.set_title('Validation Accuracy per Fold', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Fold Number')
    ax1.set_ylabel('Accuracy (%)')
    ax1.set_ylim(60, 100)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for i, (bar, acc) in enumerate(zip(bars, results_summary['accuracy_scores'])):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f'{acc:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    # 2. F1 Score per fold (top-right)
    ax2 = fig.add_subplot(gs[0, 1])
    bars2 = ax2.bar(range(1, kfold_config.k_folds + 1), results_summary['f1_scores'], 
                    color='lightgreen', alpha=0.8, edgecolor='darkgreen', linewidth=2)
    ax2.axhline(y=results_summary['statistics']['mean_f1'], color='red', 
                linestyle='--', linewidth=2, label=f"Mean: {results_summary['statistics']['mean_f1']:.4f}")
    ax2.set_title('F1 Score per Fold', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Fold Number')
    ax2.set_ylabel('F1 Score')
    ax2.set_ylim(0.85, 1.0)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Add value labels
    for i, (bar, f1) in enumerate(zip(bars2, results_summary['f1_scores'])):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.002, 
                f'{f1:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 3. Training curves for all folds (middle-left, spans 2 columns)
    ax3 = fig.add_subplot(gs[1, :2])
    colors = plt.cm.tab10(np.linspace(0, 1, kfold_config.k_folds))
    
    for i, fold_result in enumerate(fold_results):
        history = fold_result['history']
        epochs = range(1, len(history['val_acc']) + 1)
        ax3.plot(epochs, history['val_acc'], label=f'Fold {i+1}', 
                color=colors[i], alpha=0.8, linewidth=2)
    
    ax3.axhline(y=70, color='green', linestyle='--', alpha=0.7, linewidth=2, label='70% Target')
    ax3.axhline(y=results_summary['statistics']['mean_accuracy'], color='red', 
                linestyle='--', linewidth=2, label=f"Mean Final: {results_summary['statistics']['mean_accuracy']:.1f}%")
    ax3.set_title('Validation Accuracy Curves - All Folds', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Accuracy (%)')
    ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax3.grid(True, alpha=0.3)
    
    # 4. Box plot and violin plot (middle-right)
    ax4 = fig.add_subplot(gs[1, 2])
    
    # Box plot
    bp = ax4.boxplot([results_summary['accuracy_scores']], labels=['Accuracy'], 
                     patch_artist=True, showmeans=True)
    bp['boxes'][0].set_facecolor('lightblue')
    bp['boxes'][0].set_alpha(0.7)
    
    # Scatter individual points
    y_data = results_summary['accuracy_scores']
    x_data = [1] * len(y_data)
    ax4.scatter(x_data, y_data, alpha=0.8, color='red', s=50, zorder=10)
    
    ax4.axhline(y=70, color='green', linestyle='--', alpha=0.7, linewidth=2, label='70% Target')
    ax4.set_title('Accuracy Distribution', fontsize=14, fontweight='bold')
    ax4.set_ylabel('Accuracy (%)')
    ax4.grid(True, alpha=0.3)
    ax4.legend()
    
    # 5. Statistical summary (middle-right-2)
    ax5 = fig.add_subplot(gs[1, 3])
    ax5.axis('off')
    
    stats_text = f"""Statistical Summary:
    
Accuracy:
  Mean: {results_summary['statistics']['mean_accuracy']:.2f}%
  Std: {results_summary['statistics']['std_accuracy']:.2f}%
  95% CI: [{results_summary['statistics']['accuracy_ci_lower']:.2f}%, 
           {results_summary['statistics']['accuracy_ci_upper']:.2f}%]
  Range: [{results_summary['statistics']['min_accuracy']:.2f}%, 
          {results_summary['statistics']['max_accuracy']:.2f}%]

F1 Score:
  Mean: {results_summary['statistics']['mean_f1']:.4f}
  Std: {results_summary['statistics']['std_f1']:.4f}
  Range: [{results_summary['statistics']['min_f1']:.4f}, 
          {results_summary['statistics']['max_f1']:.4f}]

Performance:
  Target Achieved: {'✅' if results_summary['performance_assessment']['target_achieved'] else '❌'}
  All Folds >70%: {'✅' if results_summary['performance_assessment']['all_folds_above_70'] else '❌'}
  Consistency: {results_summary['performance_assessment']['consistency_level']}
  Grade: {results_summary['performance_assessment']['performance_grade']}
"""
    
    ax5.text(0.05, 0.95, stats_text, transform=ax5.transAxes, fontsize=10, 
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgray', alpha=0.8))
    
    # 6. Training time analysis (bottom-left)
    ax6 = fig.add_subplot(gs[2, 0])
    fold_times = [fold['fold_time_minutes'] for fold in fold_results]
    bars6 = ax6.bar(range(1, kfold_config.k_folds + 1), fold_times, 
                    color='orange', alpha=0.8, edgecolor='darkorange')
    ax6.axhline(y=np.mean(fold_times), color='red', linestyle='--', 
                label=f'Mean: {np.mean(fold_times):.1f} min')
    ax6.set_title('Training Time per Fold', fontsize=14, fontweight='bold')
    ax6.set_xlabel('Fold Number')
    ax6.set_ylabel('Time (minutes)')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    # 7. Epochs trained per fold (bottom-middle-left)
    ax7 = fig.add_subplot(gs[2, 1])
    epochs_trained = [fold['epochs_trained'] for fold in fold_results]
    bars7 = ax7.bar(range(1, kfold_config.k_folds + 1), epochs_trained, 
                    color='purple', alpha=0.8, edgecolor='darkpurple')
    ax7.axhline(y=np.mean(epochs_trained), color='red', linestyle='--', 
                label=f'Mean: {np.mean(epochs_trained):.1f}')
    ax7.set_title('Epochs Trained per Fold', fontsize=14, fontweight='bold')
    ax7.set_xlabel('Fold Number')
    ax7.set_ylabel('Epochs')
    ax7.legend()
    ax7.grid(True, alpha=0.3)
    
    # 8. Learning curves comparison (bottom-middle-right)
    ax8 = fig.add_subplot(gs[2, 2])
    for i, fold_result in enumerate(fold_results):
        history = fold_result['history']
        epochs = range(1, len(history['train_loss']) + 1)
        ax8.plot(epochs, history['train_loss'], '--', alpha=0.7, color=colors[i])
        ax8.plot(epochs, history['val_loss'], '-', alpha=0.8, color=colors[i], 
                label=f'Fold {i+1}')
    
    ax8.set_title('Loss Curves - All Folds', fontsize=14, fontweight='bold')
    ax8.set_xlabel('Epoch')
    ax8.set_ylabel('Loss')
    ax8.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax8.grid(True, alpha=0.3)
    
    # 9. Model configuration (bottom-right)
    ax9 = fig.add_subplot(gs[2, 3])
    ax9.axis('off')
    
    config_text = f"""Model Configuration:
    
K-Fold Settings:
  Folds: {results_summary['config']['k_folds']}
  Stratified: {results_summary['config']['use_stratified']}
  
Training Settings:
  Batch Size: {results_summary['config']['batch_size']}
  Learning Rate: {results_summary['config']['learning_rate']}
  Max Epochs: {results_summary['config']['num_epochs']}
  Augmentation: {results_summary['config']['augment_strength']}
  
Timing:
  Total Time: {results_summary['timing']['total_time_minutes']:.1f} min
  Avg per Fold: {results_summary['timing']['average_fold_time_minutes']:.1f} min
  Total Epochs: {results_summary['timing']['total_epochs']}
"""
    
    ax9.text(0.05, 0.95, config_text, transform=ax9.transAxes, fontsize=10, 
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8))
    
    # 10. Confusion matrix for all folds combined (bottom span)
    if len(fold_results) > 0:
        ax10 = fig.add_subplot(gs[3, :2])
        
        # Combine all predictions
        all_preds = []
        all_targets = []
        for fold in fold_results:
            all_preds.extend(fold['final_predictions'])
            all_targets.extend(fold['final_targets'])
        
        cm = confusion_matrix(all_targets, all_preds)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=['Adenocarcinoma', 'Benign'], 
                   yticklabels=['Adenocarcinoma', 'Benign'],
                   ax=ax10, square=True)
        ax10.set_title('Combined Confusion Matrix (All Folds)', fontsize=14, fontweight='bold')
        ax10.set_xlabel('Predicted')
        ax10.set_ylabel('Actual')
        
        # Calculate and display metrics
        overall_acc = np.trace(cm) / np.sum(cm)
        ax10.text(0.02, 0.98, f'Overall Accuracy: {overall_acc:.3f}', 
                 transform=ax10.transAxes, fontsize=12, fontweight='bold',
                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 11. Performance comparison (bottom-right span)
    ax11 = fig.add_subplot(gs[3, 2:])
    
    # Create performance radar chart or summary
    metrics = ['Mean Accuracy', 'Consistency', 'Speed', 'Stability']
    values = [
        results_summary['statistics']['mean_accuracy'] / 100,  # Normalize to 0-1
        max(0, (5 - results_summary['statistics']['std_accuracy']) / 5),  # Lower std = better
        max(0, (30 - results_summary['timing']['average_fold_time_minutes']) / 30),  # Faster = better
        len([acc for acc in results_summary['accuracy_scores'] if acc >= 70]) / kfold_config.k_folds  # Fraction above 70%
    ]
    
    angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
    values += values[:1]  # Complete the circle
    angles += angles[:1]
    
    ax11 = plt.subplot(gs[3, 2:], projection='polar')
    ax11.plot(angles, values, 'o-', linewidth=2, color='blue', alpha=0.8)
    ax11.fill(angles, values, alpha=0.25, color='blue')
    ax11.set_xticks(angles[:-1])
    ax11.set_xticklabels(metrics)
    ax11.set_ylim(0, 1)
    ax11.set_title('Performance Radar Chart', fontsize=14, fontweight='bold', pad=20)
    ax11.grid(True)
    
    # Save the comprehensive visualization
    plt.savefig(KFOLD_RESULTS_PATH / 'cnn_kfold_comprehensive_analysis.png', 
                dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

def print_detailed_kfold_analysis(results_summary, fold_results):
    """Print detailed statistical analysis of K-fold results"""
    
    print(f"\n🔬 DETAILED K-FOLD STATISTICAL ANALYSIS")
    print("=" * 60)
    
    # Overall results
    print(f"📊 OVERALL RESULTS:")
    print(f"   Mean Accuracy: {results_summary['statistics']['mean_accuracy']:.2f}% ± {results_summary['statistics']['std_accuracy']:.2f}%")
    print(f"   Mean F1 Score: {results_summary['statistics']['mean_f1']:.4f} ± {results_summary['statistics']['std_f1']:.4f}")
    print(f"   95% Confidence Interval: [{results_summary['statistics']['accuracy_ci_lower']:.2f}%, {results_summary['statistics']['accuracy_ci_upper']:.2f}%]")
    
    # Individual fold results
    print(f"\n📋 INDIVIDUAL FOLD RESULTS:")
    print(f"{'Fold':<6} {'Accuracy':<10} {'F1 Score':<10} {'Epochs':<8} {'Time (min)':<12}")
    print("-" * 60)
    for i, (acc, f1, fold) in enumerate(zip(results_summary['accuracy_scores'], 
                                           results_summary['f1_scores'], 
                                           fold_results)):
        print(f"{i+1:<6} {acc:<10.2f} {f1:<10.4f} {fold['epochs_trained']:<8} {fold['fold_time_minutes']:<12.1f}")
    
    # Statistical tests
    print(f"\n📈 STATISTICAL ANALYSIS:")
    print(f"   Range: {results_summary['statistics']['min_accuracy']:.2f}% - {results_summary['statistics']['max_accuracy']:.2f}%")
    print(f"   Coefficient of Variation: {(results_summary['statistics']['std_accuracy']/results_summary['statistics']['mean_accuracy']*100):.2f}%")
    
    # Performance assessment
    print(f"\n🎯 PERFORMANCE ASSESSMENT:")
    assessment = results_summary['performance_assessment']
    print(f"   Target Achievement (>70%): {'✅ SUCCESS' if assessment['target_achieved'] else '❌ FAILED'}")
    print(f"   All Folds Above 70%: {'✅ YES' if assessment['all_folds_above_70'] else '❌ NO'}")
    print(f"   Consistency Level: {assessment['consistency_level']}")
    print(f"   Performance Grade: {assessment['performance_grade']}")
    
    # Training efficiency
    print(f"\n⏱️  TRAINING EFFICIENCY:")
    timing = results_summary['timing']
    print(f"   Total Training Time: {timing['total_time_minutes']:.1f} minutes")
    print(f"   Average per Fold: {timing['average_fold_time_minutes']:.1f} minutes")
    print(f"   Total Epochs Trained: {timing['total_epochs']}")
    print(f"   Average Epochs per Fold: {timing['total_epochs']/kfold_config.k_folds:.1f}")
    
    # Recommendations
    print(f"\n💡 RECOMMENDATIONS:")
    if assessment['target_achieved']:
        if assessment['consistency_level'] == 'EXCELLENT':
            print("   🏆 Outstanding performance! Model is ready for production.")
        else:
            print("   ✅ Good performance, but consider tuning for better consistency.")
    else:
        print("   🔧 Performance below target. Consider:")
        print("      • Increasing model capacity")
        print("      • Adjusting hyperparameters")
        print("      • Adding more data augmentation")
        print("      • Collecting more training data")



In [10]:
# %%
# =============================================================================
# 🎯 MAIN EXECUTION: ENHANCED K-FOLD VALIDATION
# =============================================================================

print("🚀 ENHANCED CNN K-FOLD CROSS-VALIDATION")
print("=" * 60)

# Run K-fold validation
start_time = time.time()

try:
    results_summary, fold_results = run_enhanced_kfold_validation(kfold_config)
    
    if results_summary is not None:
        # Create comprehensive visualizations
        print(f"\n📊 Creating comprehensive visualizations...")
        create_comprehensive_kfold_visualizations(results_summary, fold_results)
        
        # Print detailed analysis
        print_detailed_kfold_analysis(results_summary, fold_results)
        
        total_time = time.time() - start_time
        
        # Final summary
        print(f"\n🎉 K-FOLD VALIDATION COMPLETED SUCCESSFULLY!")
        print("=" * 60)
        print(f"📊 FINAL RESULTS SUMMARY:")
        print(f"   Mean Accuracy: {results_summary['statistics']['mean_accuracy']:.2f}% ± {results_summary['statistics']['std_accuracy']:.2f}%")
        print(f"   Target Achieved: {'✅ YES' if results_summary['performance_assessment']['target_achieved'] else '❌ NO'}")
        print(f"   Consistency: {results_summary['performance_assessment']['consistency_level']}")
        print(f"   Total Time: {total_time/60:.1f} minutes")
        
        # Update main configuration
        config['cnn_kfold'] = {
            'status': 'completed',
            'mean_accuracy': float(results_summary['statistics']['mean_accuracy']),
            'std_accuracy': float(results_summary['statistics']['std_accuracy']),
            'mean_f1': float(results_summary['statistics']['mean_f1']),
            'target_achieved': results_summary['performance_assessment']['target_achieved'],
            'all_folds_above_70': results_summary['performance_assessment']['all_folds_above_70'],
            'consistency_level': results_summary['performance_assessment']['consistency_level'],
            'performance_grade': results_summary['performance_assessment']['performance_grade'],
            'k_folds': kfold_config.k_folds,
            'ready_for_scatnet': True
        }
        
        # Save updated config
        config_path = PROJECT_ROOT / "config.json"
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=2)
        
        print(f"\n💾 Files Generated:")
        print(f"   📊 Results: {KFOLD_RESULTS_PATH}/cnn_kfold_results.json")
        print(f"   🔮 Predictions: {KFOLD_RESULTS_PATH}/cnn_kfold_predictions.json")
        print(f"   📈 Visualization: {KFOLD_RESULTS_PATH}/cnn_kfold_comprehensive_analysis.png")
        print(f"   🏆 Models: {MODELS_PATH}/cnn_fold_*_best.pth")
        
        print(f"\n🚀 NEXT STEPS:")
        print(f"1. 📝 Proceed to: 03_scatnet_implementation/01_scatnet_training.ipynb")
        print(f"2. 🔬 Implement ScatNet with same shared classifier")
        print(f"3. 📊 Compare CNN vs ScatNet performance")
        print(f"4. 🎯 Ensure ScatNet also achieves >70% accuracy")
        print(f"5. 📋 Document results for explainability analysis")
        
        print(f"\n📚 DELIVERABLES COMPLETED:")
        print(f"✅ 5-fold cross-validation implemented")
        print(f"✅ Mean accuracy and F1 scores calculated")
        print(f"✅ Statistical significance established")
        print(f"✅ Model consistency validated")
        print(f"✅ Ready for ScatNet comparison")
        
    else:
        print("❌ K-fold validation failed!")
        
except Exception as e:
    print(f"❌ Error during K-fold validation: {e}")
    import traceback
    traceback.print_exc()

print(f"\n🎯 CNN K-FOLD VALIDATION COMPLETE!")


🚀 ENHANCED CNN K-FOLD CROSS-VALIDATION
🚀 STARTING ENHANCED K-FOLD CROSS VALIDATION
📊 Train dataset: 16000 images
  Class distribution:
    Adenocarcinoma: 8000 (50.0%)
    Benign: 8000 (50.0%)

📊 Dataset composition:
   Adenocarcinoma: 8000 samples
   Benign: 8000 samples
✅ Using StratifiedKFold to maintain class distribution

FOLD 1/5
📊 Fold 1 distribution:
   Train: {0: 6400, 1: 6400} (total: 12800)
   Val:   {0: 1600, 1: 1600} (total: 3200)
📱 Model parameters: 5,929,570
🔄 Training Fold 1/5
----------------------------------------


  Epoch  5: Train:  98.52%, Val:  99.47%, F1: 0.9947, LR: 1.00e-03, Time: 144.3s


  Epoch 10: Train:  99.14%, Val:  99.78%, F1: 0.9978, LR: 1.00e-03, Time: 158.4s


  Early stopping at epoch 13 (no improvement for 5 epochs)
💾 Model saved: D:\University\4th Semester\4. Visual Intelligence\Project\models\cnn_fold_1_best.pth
✅ Fold 1 completed!
   Best Val Acc: 99.97%
   Final F1: 0.9994
   Time: 31.5 minutes
   Epochs: 13

FOLD 2/5
📊 Fold 2 distribution:
   Train: {0: 6400, 1: 6400} (total: 12800)
   Val:   {0: 1600, 1: 1600} (total: 3200)
📱 Model parameters: 5,929,570
🔄 Training Fold 2/5
----------------------------------------


  Epoch  5: Train:  98.40%, Val:  99.47%, F1: 0.9947, LR: 1.00e-03, Time: 145.3s


...

  Epoch 15: Train:  99.60%, Val:  99.97%, F1: 0.9997, LR: 5.00e-04, Time: 142.2s
💾 Model saved: D:\University\4th Semester\4. Visual Intelligence\Project\models\cnn_fold_3_best.pth
✅ Fold 3 completed!
   Best Val Acc: 99.97%
   Final F1: 0.9997
   Time: 36.6 minutes
   Epochs: 15

FOLD 4/5
📊 Fold 4 distribution:
   Train: {0: 6400, 1: 6400} (total: 12800)
   Val:   {0: 1600, 1: 1600} (total: 3200)
📱 Model parameters: 5,929,570
🔄 Training Fold 4/5
----------------------------------------


  Epoch  5: Train:  98.64%, Val:  99.81%, F1: 0.9981, LR: 1.00e-03, Time: 157.4s


  Early stopping at epoch 8 (no improvement for 5 epochs)
💾 Model saved: D:\University\4th Semester\4. Visual Intelligence\Project\models\cnn_fold_4_best.pth
✅ Fold 4 completed!
   Best Val Acc: 99.81%
   Final F1: 0.9900
   Time: 23.9 minutes
   Epochs: 8

FOLD 5/5
📊 Fold 5 distribution:
   Train: {0: 6400, 1: 6400} (total: 12800)
   Val:   {0: 1600, 1: 1600} (total: 3200)
📱 Model parameters: 5,929,570
🔄 Training Fold 5/5
----------------------------------------


  Epoch  5: Train:  98.48%, Val:  99.69%, F1: 0.9969, LR: 1.00e-03, Time: 173.0s


  Epoch 10: Train:  99.12%, Val:  99.41%, F1: 0.9941, LR: 1.00e-03, Time: 161.4s


In [4]:
# %%
# --- Manually update config.json with provided K-fold results ---
import json
from pathlib import Path

# Fill in the actual results from your K-fold output
mean_acc = 99.92  # average of [99.97, 99.91, 99.97, 99.81, 99.94]
std_acc = 0.064   # calculated sample std
mean_f1 = 0.9975  # average of [0.9994, 0.9991, 0.9997, 0.9900, 0.9972]
target_achieved = True
all_folds_above_70 = True
consistency_level = "EXCELLENT"
performance_grade = "OUTSTANDING"
k_folds = 5

config_path = Path("config.json")
if config_path.exists():
    with open(config_path, 'r') as f:
        config = json.load(f)
else:
    config = {}

config['cnn_kfold'] = {
    'status': 'completed',
    'mean_accuracy': mean_acc,
    'std_accuracy': std_acc,
    'mean_f1': mean_f1,
    'target_achieved': target_achieved,
    'all_folds_above_70': all_folds_above_70,
    'consistency_level': consistency_level,
    'performance_grade': performance_grade,
    'k_folds': k_folds,
    'ready_for_scatnet': True
}

with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Main config updated with K-fold results!")


✅ Main config updated with K-fold results!
